In [97]:
# !pip install spotipy

In [98]:
# !pip install kneed

In [99]:
# importing necessary packages
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from kneed import KneeLocator
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

In [100]:
# loading our dataset downloaded from kaggle
df = pd.read_csv("data.csv")
df.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [101]:
print(f"Shape: {df.shape}\n")

print(f"Dupes?: {df.duplicated().sum()}\n")

print(f"Nulls?: \n{df.isnull().sum()}")

Shape: (170653, 19)

Dupes?: 0

Nulls?: 
valence             0
year                0
acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
explicit            0
id                  0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
name                0
popularity          0
release_date        0
speechiness         0
tempo               0
dtype: int64


In [102]:
df.columns # column names

Index(['valence', 'year', 'acousticness', 'artists', 'danceability',
       'duration_ms', 'energy', 'explicit', 'id', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'name', 'popularity', 'release_date',
       'speechiness', 'tempo'],
      dtype='object')

In [103]:
df.dtypes # data types

valence             float64
year                  int64
acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
release_date         object
speechiness         float64
tempo               float64
dtype: object

  ##### Information on the Columns
  * **Valence**: Measure from 0.0 to 1.0 that describes positivity in tracks.
  * **Year**: The year in which the track was released.
  * **Acousticness**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic, higher value is better confidence.
  * **Artists**: Names of the Artists.
  * **Danceability**: Describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
  * **Duration**: Length of song in microseconds
  * **Energy**: a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
  * **Explicit**: Value 1 = Explicit, Value 0 = Not Explicit
  * **id**: Song ID via Spotify
  * **Instrumentalness**: Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
  * **Key**: The estimated overall key of the track. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.
  * **Loudness**: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.
  * **Mode**: Indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
  * **Popularity**: The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time.
  * **Speechiness**: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
  * **Tempo**: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

In [104]:
# dropping unnecessary columns
df_features = df.drop(columns=['id','name','release_date','explicit','artists','popularity','year','duration_ms'], axis=1)
display(df_features.head(10))

,valence,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo
0,0.0594,0.982,0.279,0.211,0.878000,10,0.665,-20.096,1,0.0366,80.954
1,0.9630,0.732,0.819,0.341,0.000000,7,0.160,-12.441,1,0.4150,60.936
2,0.0394,0.961,0.328,0.166,0.913000,3,0.101,-14.850,1,0.0339,110.339
3,0.1650,0.967,0.275,0.309,0.000028,5,0.381,-9.316,1,0.0354,100.109
4,0.2530,0.957,0.418,0.193,0.000002,3,0.229,-10.096,1,0.0380,101.665
5,0.1960,0.579,0.697,0.346,0.168000,2,0.130,-12.506,1,0.0700,119.824
6,0.4060,0.996,0.518,0.203,0.000000,0,0.115,-10.589,1,0.0615,66.221
7,0.0731,0.993,0.389,0.088,0.527000,1,0.363,-21.091,0,0.0456,92.867
8,0.7210,0.996,0.485,0.130,0.151000,5,0.104,-21.508,0,0.0483,64.678
9,0.7710,0.982,0.684,0.257,0.000000,8,0.504,-16.415,1,0.3990,109.378


In [105]:
df_features.columns

Index(['valence', 'acousticness', 'danceability', 'energy', 'instrumentalness',
       'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo'],
      dtype='object')

In [106]:
# utilizing spotify API with 2 keys
SPOTIFY_KEY1 = '798c070d2d5e4ab98b36353e469dba19'
SPOTIFY_KEY2 = 'ec5f36a15c864212a84ab03d15fc7c74'

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(SPOTIFY_KEY1,SPOTIFY_KEY2))


In [107]:
# print function from main.py to show input track info
def print_info(track):
    print('\nArtist: ' + track['artists'][0]['name'])
    print('Track: ' + track['name'])
    print('Album: ' + track['album']['name'])
    AP = track['preview_url']
    if AP is None:
        print("No Audio Preview available")
    else:
        print('Audio Preview: ' + track['preview_url'])
    print('Cover Art: ' + track['album']['images'][0]['url'])

In [108]:
# accepts input uri from spotify
found = False
while not found:
    track_id = input("Enter uri: ")
    if spotify.track(track_id):
        found = True

track = spotify.track(track_id)

song_features = spotify.audio_features(track_id)


Enter uri: spotify:track:51hyZpbJlIgZIaX3TtMxmu


In [109]:
print_info(track)
print(song_features)


Artist: Tate McRae
Track: run for the hills
Album: THINK LATER
Audio Preview: https://p.scdn.co/mp3-preview/0c91e2da7c4ccc1f9c9e7de651f867e7e9805b38?cid=798c070d2d5e4ab98b36353e469dba19
Cover Art: https://i.scdn.co/image/ab67616d0000b27377eb7a5b9367de79c3b92a49
[{'danceability': 0.544, 'energy': 0.831, 'key': 8, 'loudness': -3.578, 'mode': 1, 'speechiness': 0.0941, 'acousticness': 0.543, 'instrumentalness': 0, 'liveness': 0.118, 'valence': 0.233, 'tempo': 130.005, 'type': 'audio_features', 'id': '51hyZpbJlIgZIaX3TtMxmu', 'uri': 'spotify:track:51hyZpbJlIgZIaX3TtMxmu', 'track_href': 'https://api.spotify.com/v1/tracks/51hyZpbJlIgZIaX3TtMxmu', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/51hyZpbJlIgZIaX3TtMxmu', 'duration_ms': 143940, 'time_signature': 4}]


In [110]:
# input track information
df_song = pd.DataFrame(song_features)
df_song = df_song.drop(columns=['track_href','analysis_url','type','id','uri','time_signature'])
print(df_song)


   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.544   0.831    8    -3.578     1       0.0941         0.543   

   instrumentalness  liveness  valence    tempo  duration_ms  
0                 0     0.118    0.233  130.005       143940  


In [111]:
X_train = df_features[['valence', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo']]
X_train



,valence,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo
0,0.0594,0.98200,0.279,0.211,0.878000,10,0.6650,-20.096,1,0.0366,80.954
1,0.9630,0.73200,0.819,0.341,0.000000,7,0.1600,-12.441,1,0.4150,60.936
2,0.0394,0.96100,0.328,0.166,0.913000,3,0.1010,-14.850,1,0.0339,110.339
3,0.1650,0.96700,0.275,0.309,0.000028,5,0.3810,-9.316,1,0.0354,100.109
4,0.2530,0.95700,0.418,0.193,0.000002,3,0.2290,-10.096,1,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...
170648,0.6080,0.08460,0.786,0.808,0.000289,7,0.0822,-3.702,1,0.0881,105.029
170649,0.7340,0.20600,0.717,0.753,0.000000,7,0.1010,-6.020,1,0.0605,137.936
170650,0.6370,0.10100,0.634,0.858,0.000009,4,0.2580,-2.226,0,0.0809,91.688
170651,0.1950,0.00998,0.671,0.623,0.000008,2,0.6430,-7.161,1,0.3080,75.055


In [112]:
# normalize the variables since variance is range dependent (if variables have a high range, it may skew our results)
normalized_df = normalize(df_features)

In [113]:
# save the result in a data frame called data_scaled
df_scaled = pd.DataFrame(normalized_df)
# .var() function calculates the variance
variance = df_scaled.var()

In [114]:
variance

0     0.000006
1     0.000017
2     0.000004
3     0.000007
4     0.000012
5     0.001282
6     0.000004
7     0.005072
8     0.000023
9     0.000003
10    0.000977
dtype: float64

In [115]:
#storing name of variables
columns = df_features.columns

In [116]:
# set threshold of 0.00005. If the variable has a variance greater than a threshold, we will select it and drop the rest
# saving the names of variables having variance more than a threshold value

variable = [ ]

for i in range(0,len(variance)):
    if variance[i]>=0.000005: #setting the threshold as 0.0006
        variable.append(columns[i])

In [117]:
variable

['valence',
 'acousticness',
 'energy',
 'instrumentalness',
 'key',
 'loudness',
 'mode',
 'tempo']

In [118]:
# creating a new dataframe using the above variables (after low variance filter)
df_new = df_features[variable]

# first five rows of the new data
df_new.head()

,valence,acousticness,energy,instrumentalness,key,loudness,mode,tempo
0,0.0594,0.982,0.211,0.878000,10,-20.096,1,80.954
1,0.9630,0.732,0.341,0.000000,7,-12.441,1,60.936
2,0.0394,0.961,0.166,0.913000,3,-14.850,1,110.339
3,0.1650,0.967,0.309,0.000028,5,-9.316,1,100.109
4,0.2530,0.957,0.193,0.000002,3,-10.096,1,101.665


In [119]:
# compare shape of new and original data
df_new.shape, df_features.shape

((170653, 8), (170653, 11))

In [120]:
# new input track info (after low variance filter)
df_song = df_song[variable]
df_song

,valence,acousticness,energy,instrumentalness,key,loudness,mode,tempo
0,0.233,0.543,0.831,0,8,-3.578,1,130.005


In [122]:
# utilizing K-means clustering technique
kmeans_model = KMeans(n_clusters=10)

kmeans_model.fit(df_new)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=10)

In [123]:
clustered = df.copy()
clustered['type'] = kmeans_model.labels_

# creating predictions using Kmeans model
predictions = kmeans_model.predict(df_song)

# creating recommendations (filter to top 5)
recommendations = clustered[clustered.type == predictions[0]]
recommendations.drop_duplicates()
recommendations = recommendations.sample(n=5)

#only show artist, name, and id for recommendations
recommendations = recommendations[['artists', 'name', 'id']]

In [124]:
recommendations

,artists,name,id
27957,['Jay & The Techniques'],"Apples, Peaches, Pumpkin Pie",1m1UTLYGCzEciEdSfpK1Yu
86170,['KISS'],Rise To It,7gAHDwN4iqkjFozSODtWBn
90687,['Fidel Rueda'],Tú Ya Eres Cosa Del Pasado,781lqqMzOTUPNl21JdgaLS
127953,['Hampton Hawes Trio'],Steeplechase,1iJ8QjC6ROpF0zZz4S8zrE
30719,['Roxy Music'],Oh Yeah!,1ApsBaz9fpPptVZxgqtOy3


In [124]:
def kmeans_lvf_model(df,uisDATA):
  features = ['valence','acousticness','energy','instrumentalness','key','loudness','mode','tempo']
  df_lvf = df[features]
  df_song = uisDATA[features]

  kmeans_model = KMeans(n_clusters=10)
  kmeans_model.fit(df_lvf)

  clustered = df.copy()
  clustered['type'] = kmeans_model.labels_

  predictions = kmeans_model.predict(df_song)

  recommendations = clustered[clustered.type == predictions[0]]
  recommendations.drop_duplicates(inplace=True)
  recommendations = recommendations.sample(n=5)
  recommendations = recommendations[['artists', 'name', 'id']]

  return recommendations

In [125]:
# # n_neighbors as kneighbors function returns distance of point to itself
# neigh = NearestNeighbors(n_neighbors=22).fit(df_features)

In [126]:
# # find the k-neighbors of a point
# neigh_dist, neigh_ind = neigh.kneighbors(df_features)

In [127]:
# # sort the neighbors distances (lengths to points) in ascending order
# # axis = 0 represents sort along first axis (sort along row)
# sort_neigh_dist =  np.sort(neigh_dist, axis=0)

In [128]:
# # plot the KNN distance plot to find eps parameter
# k_dist = sort_neigh_dist[:, 4]
# plt.plot(k_dist)
# plt.rcParams["figure.figsize"] = (4,4)
# plt.ylabel("K-NN Distance")
# plt.xlabel("Sorted Observations (4th NN)")
# plt.show

In [129]:
# # using KneeLocator to estimate the "knee point" or the threshold value where there is a sharp change in the K-NN distance plot
# kneedle = KneeLocator(x=range(1,len(neigh_dist)+1), y=k_dist, S=1.0, curve="concave", direction="increasing", online=True)

# print(kneedle.knee_y)
# # the optimal eps parameter is 8.92

In [130]:
# # show the knee point estimate on the K-NN distance plot
# kneedle.plot_knee()
# plt.show()

In [131]:
# clusters = DBSCAN(eps=8.92, min_samples=22).fit(df_features)
# # DBSCAN_dataset = df_features.copy()
# # DBSCAN_dataset.loc[:,'Cluster'] = clustering.labels_